# W2D1 Tutorial 4: Return levels using Normal and GEV distributions

#### __Week 2, Day 4, Extremes & Vulnerability__
##### __Content creators:__ Matthias Aengenheyster, Joeri Reinders
##### __Content reviewers:__ TBD
##### __Content editors:__ TBD
##### __Production editors:__ TBD
##### __Our 2023 Sponsors:__ TBD

## Tutorial Objectives:

Now that we know how to create pdf for the GEV distribution we can use it to compute return values 
based on a GEV distribution.

We  compute return levels from a pdf by using the cumulative distribution function (CDF), which tells us
the chance that a randomly drawn variable from our distribution will be less than some value X (for our 
variable). Rare extreme values thus have a high chance (as most random draws will be smaller – small storms).
Again, say that there is a 99% chance that we will see a storm of 80mm or higher… than there is a 1% change
we will see a storm of at least 80mm – and given what we said above it means the 100-year storm will bring 
80mm of rain. Formally the return level is the inverse of the CDF – for here all you need to know is that 
you can compute return-levels using the function estimate_return_level_from_params(x,location,scale,shape). In this case x is the chance
a random value from our distribution is lower – so for the 100-year storm this would be 0.99; and for the
1000-year storm 0.999.  

By the end of the tutorial you will be able to:
- Estimate return levels given a quantile and the parameters of a GEV distribution
- Compare return levels plots for GEV and normal distributions

## Setup

In [ ]:
# Installs

In [ ]:
# # <Yosmely Bermúdez> comments
# #install dependencies
# # We need this to install eigen which is needed for SDFC to install correctly
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install eigen
# !pip install -v https://github.com/yrobink/SDFC/archive/master.zip#subdirectory=python
# !pip install https://github.com/njleach/mystatsfunctions/archive/master.zip
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install pandas
# !pip install cartopy
# !pip install scipy
# !pip install texttable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy import stats
gev = stats.genextreme

import extremes_functions as ef
from mystatsfunctions import OLSE,LMoments
import SDFC as sd

In [ ]:
import os, pooch

fname = 'precipitationGermany_1920-2022.csv'
if not os.path.exists(fname):
    url = "https://osf.io/xs7h6/download"
    fname = pooch.retrieve(url, known_hash=None)

data = pd.read_csv(fname, index_col=0).set_index('years')

data.columns=['precipitation']
precipitation = data.precipitation

In [ ]:
fit, model = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True)

In [ ]:
fit

In [ ]:
model

This utility function takes a given *quantile* and a fitted *model* and computes the corresponding return level

In [ ]:
def estimate_return_level(quantile,model):
    loc, scale, shape = model.coef_
    level = loc - scale / shape * (1 - (-np.log(quantile))**(-shape))
    # level = stats.genextreme.ppf(quantile,-shape,loc=loc,scale=scale)
    return level

def estimate_return_level_from_params(quantile,loc,scale,shape):
    level = loc - scale / shape * (1 - (-np.log(quantile))**(-shape))
    return level

Now let's use this function to compute the 2-year level, which is the level of precipitation we expect with a 50% chance each year. The function needs a quantile x, and the three GEV parameters: location, scale, and shape. The output of the function is the return level at that quantile. 
For example the 90th quantile (x=0.9) would give the 10-year precipitation value (10% change of seeing it every year).

In [ ]:
estimate_return_level(0.5,model)

In [ ]:
stats.genextreme.ppf(0.5,-model.shape.mean(),loc=model.loc.mean(),scale=model.scale.mean())

It is probably easier to replace x with a variable called “rp” (= return period). You can use the formula 1-(1/return period)
to compute rp. So a 100-year storm would be 1-(1/100) = 0.99. Compute the 100- and 1000-year storm. 

In [ ]:
estimate_return_level(1-1/80,model)

In [ ]:
estimate_return_level(1-1/1000,model)

Now we will make a figure in which we plot return-levels against return-periods:
1. Create vector “periods” which is a sequence of 2 to a 1000-years (in steps of 2).
2. Create a vector “levels” which is the associated return-levels for those return periods.
3. Plot them against each other. Typically the return periods go on the x-axis at a log-scale.


In [ ]:
periods = np.arange(2,1000,2)
quantiles = 1-1/periods
levels = estimate_return_level(quantiles,model)
# plt.plot(x,estimate_return_level(x,model))
plt.plot(periods,levels,'.-')
plt.xlabel('Return Period (years)')
plt.ylabel('Return level')

plt.semilogx()

It would be interesting to compare the return levels from a normal and GEV distribution. Compute the 
50-,100-,500- and 1000-year based on a normal distribution and based on the GEV distribution
like above. 

You can use the stats.norm.ppf and stats.genextreme.ppf functions: The "Percent point function" (ppf) is the inverse of the CDF function: It takes the quantiles (and the distribution parameters) and returns the values. 
For the normal distribution, you need mean and standard deviation. 
For the GEV distribution, you need shape, location, and scale. Note that there are two sign conventions for the shape parameter - You will have to multiply the shape from the fit above by -1.

In [ ]:
years = np.array([50,100,500,1000])
quantiles = 1-1/years

df = pd.DataFrame(index=years)
df['Normal Distribution'] = stats.norm.ppf(quantiles,loc=precipitation.mean(), scale=precipitation.std())
df['GEV'] = stats.genextreme.ppf(quantiles,-model.shape.mean(),loc=model.loc.mean(),scale=model.scale.mean())

In [ ]:
df

## Excercise: Plot and compare return levels for Normal and GEV distributions
Create a return time plot that plots
1. the empirical return level
2. the estimate based on a normal distribution
3. the estimate based on a GEV distribution

Note that the empirical return level is contained in the "return_level_obs" variable of the "fit" dataset:

In [ ]:
fit.return_level_obs

### student

In [ ]:
...

### solution

In [ ]:
fit.return_level_obs.plot.line('.')
years = np.arange(1.1,100,0.1)
plt.plot(years,stats.norm.ppf(1-1/years,loc=precipitation.mean(), scale=precipitation.std()))
plt.plot(years,stats.genextreme.ppf(1-1/years,-model.shape.mean(),loc=model.loc.mean(),scale=model.scale.mean()))

plt.semilogx()

plt.legend(['empirical','normal','GEV'])

### What can you say about the plot and how the distributions describe the data? How do the distributions differ? At short/long return periods? What will happen at even longer return periods?

## Optional extra task: Implement your own bootstrapping algorithm
We have used the pre-written function ef.fit_return_levels_sdfc to fit the GEV distribution to the data, compute confidence intervals and the like. You may find it instructive to try and implement such an algorithm from scratch. 
One pathway would be the following:
1. Start from the 'gev' function (scipy.stats.genextreme). It can be used to fit by GEV parameters by stats.genextreme.fit(data), and returns the three parameters (note a sign change in the shape parameter - different conventions exist!)
2. Check how many observations there are in the data, let's call this number N.
3. Resample the data by drawing N samples out of the data, with replacing - this creates an artificial ensemble, different from the true data due to the resampling.
4. Estimate the parameters for each resampling.
5. Use the stats.genextreme.ppf for each parameter set to compute the return level for each one.
6. Visualize the resulting uncertainty bands, e.g. by plotting many lines or computing confidence intervals using np.quantile.


### solution

In [ ]:
params = []
for i in range(1000):
    params.append(gev.fit(np.random.choice(precipitation,size=precipitation.size,replace=True)))

In [ ]:
print('Mean estimate and 95% confidence intervals')
[
    np.mean(np.array(params),axis=0),
    np.quantile(np.array(params),[0.025,0.975],axis=0)
]

In [ ]:
years = np.arange(1.1,1000,0.1)
levels = []
for i in range(1000):
    levels.append(gev.ppf(1-1/years,*params[i]))
levels = np.array(levels)

In [ ]:
plt.plot(years,levels.mean(axis=0))
plt.plot(years,np.quantile(levels,[0.025,0.975],axis=0).T,'k--')
plt.semilogx()
plt.xlim(1.5,1000)
plt.ylim(20,110)
plt.xlabel('return period (years)')
plt.ylabel('return level (mm/day)')